In [30]:
# Import APIs
import numpy as np
import scipy.io

In [3]:
# Define data path
path = 'YOUR PATH
def mat_to_npy(subject):
    dr_event = scipy.io.loadmat(path  + "/sb%d_eeg.mat"%subject)["eeg"]["event"][0, 0]
    dr_data = scipy.io.loadmat(path  + "/sb%d_eeg.mat"%subject)["eeg"]["data"][0, 0]
    dr_flag = np.where(dr_event == 1)[-1][0] 
    
    dr_data = dr_data[:, dr_flag:]
    dr_tmp = np.zeros(shape=(180, 64, 5120)) # (180 trials, 64 channels, 10 second timepoints)
    for i in range(dr_tmp.shape[0]):
        dr_tmp[i, :, :] = dr_data[:64, i*5120:(i+1)*5120]
        
    wk_event = scipy.io.loadmat(path + "/sg%d_eeg.mat"%subject)["eeg"]["event"][0, 0]
    wk_data = scipy.io.loadmat(path + "/sg%d_eeg.mat"%subject)["eeg"]["data"][0, 0]
    wk_flag = np.where(wk_event == 1)[-1][0] 
        
    wk_data = wk_data[:, wk_flag:]
    wk_tmp = np.zeros(shape=(180, 64, 5120)) # (180 trials, 64 channels, 10 second timepoints)
    for i in range(wk_tmp.shape[0]):
        wk_tmp[i, :, :] = wk_data[:64, i*5120:(i+1)*5120]
    
    dr_label = np.zeros(shape=dr_tmp.shape[0])
    wk_label = np.ones(shape=wk_tmp.shape[0])
    
    
    print(dr_tmp.shape, dr_label.shape, wk_tmp.shape, wk_label.shape)
    np.save(path + "/5fold/subject%02d_drowsiness_eeg.npy"%subject, dr_tmp)
    np.save(path + "/5fold/subject%02d_drowsiness_label.npy"%subject, dr_label)
    np.save(path + "/5fold/subject%02d_normal_eeg.npy"%subject, wk_tmp)
    np.save(path + "/5fold/subject%02d_normal_label.npy"%subject, wk_label)
    return

In [5]:
def divide_5fold(subject):
    path = 'YOUR PATH    dr_eeg = np.load(path + "/subject%02d_drowsiness_eeg.npy"%subject)
    dr_lbl = np.load(path + "/subject%02d_drowsiness_label.npy"%subject)
    wk_eeg = np.load(path + "/subject%02d_normal_eeg.npy"%subject)
    wk_lbl = np.load(path + "/subject%02d_normal_label.npy"%subject)
    
    np.random.seed(970304)
    rand_idx1 = np.random.permutation(dr_eeg.shape[0])
    
    np.random.seed(5930)
    rand_idx2 = np.random.permutation(wk_eeg.shape[0])
    
    dr_eeg = dr_eeg[rand_idx1, :, :]
    dr_lbl = dr_lbl[rand_idx1]
    wk_eeg = wk_eeg[rand_idx2, :, :]
    wk_lbl = wk_lbl[rand_idx2]
    
    tmp = int(dr_eeg.shape[0]/5)
    for i in range(0, 5):
        tmp1 = dr_eeg[i*tmp:(i+1)*tmp, :, :] # drowsy test eeg
        tmp2 = dr_lbl[i*tmp:(i+1)*tmp] # drowsy test label
        tmp3 = wk_eeg[i*tmp:(i+1)*tmp, :, :] # wake test eeg
        tmp4 = wk_lbl[i*tmp:(i+1)*tmp] # wake test label
        tmp5 = np.concatenate((dr_eeg[:i*tmp, :, :], dr_eeg[(i+1)*tmp:, :, :]),axis=0) # drowsy train eeg
        tmp6 = np.concatenate((dr_lbl[:i*tmp], dr_lbl[(i+1)*tmp:]),axis=0) # drowsy train label
        tmp7 = np.concatenateenate((wk_eeg[:i*tmp, :, :], wk_eeg[(i+1)*tmp:, :, :]),axis=0) # wake train eeg
        tmp8 = np.concatenate((wk_lbl[:i*tmp], wk_lbl[(i+1)*tmp:]),axis=0) # wake train label
        
#         print(tmp1.shape, tmp2.shape, tmp3.shape, tmp4.shape)
#         print(tmp5.shape, tmp6.shape, tmp7.shape, tmp8.shape)
        
        train_eeg = np.concatenate((tmp5, tmp7), axis=0)
        train_label = np.concatenate((tmp6, tmp8), axis=0)
        
        test_eeg = np.concatenate((tmp1, tmp3), axis=0)
        test_label = np.concatenate((tmp2, tmp4), axis=0)
        
        print(train_eeg.shape, train_label.shape, test_eeg.shape, test_label.shape)
        
#         np.save(path + "/cv%01d_sbj%02d_test_eeg.npy"%(i+1, subject), test_eeg)
#         np.save(path + "/cv%01d_sbj%02d_test_label.npy"%(i+1, subject), test_label)
        
#         np.save(path + "/cv%01d_sbj%02d_train_eeg.npy"%(i+1, subject), train_eeg)
#         np.save(path + "/cv%01d_sbj%02d_train_label.npy"%(i+1, subject), train_label)
        
    return

for i in range(1, 11):
    divide_5fold(i)
    print(i)

    

(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
1
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
2
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
3
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
4
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (72, 64, 5120) (72,)
(288, 64, 5120) (288,) (

In [4]:
def divide_Scenario1_set(subject):
    
    path = 'YOUR PATH    save_path = 'YOUR PATH    
    for i in range (1,6):
        for j in range(1, subject):
            if j == 1:
                trainEEG_1 = np.load(path + "/cv%01d_sbj%02d_train_eeg.npy"%(i, j))
                trainEEG_2 = np.load(path + "/cv%01d_sbj%02d_train_eeg.npy"%(i, j+1))
                train_eeg = np.concatenate((trainEEG_1, trainEEG_2), axis=0)
                
                trainLABEL_1 = np.load(path + "/cv%01d_sbj%02d_train_label.npy"%(i, j))
                trainLABEL_2 = np.load(path + "/cv%01d_sbj%02d_train_label.npy"%(i, j+1))
                train_label = np.concatenate((trainLABEL_1, trainLABEL_2), axis=0)
                
                print(j, train_eeg.shape, train_label.shape)
                
            elif j > 2:
                trainEEG_3 = np.load(path + "/cv%01d_sbj%02d_train_eeg.npy"%(i, j))
                train_eeg = np.concatenate((train_eeg, trainEEG_3))
                
                trainLABEL_3 = np.load(path + "/cv%01d_sbj%02d_train_label.npy"%(i, j))
                train_label = np.concatenate((train_label, trainLABEL_3))
                
                print(j, train_eeg.shape, train_label.shape)

#         np.save(save_path + "/sbj%02d_%02d_train_eeg.npy"%(1, 9), train_eeg)
#         np.save(save_path + "/sbj%02d_%02d_train_label.npy"%(1, 9), train_label)
        
        print("fold number %d\n" % i)
        
divide_Scenario1_set(10)

1 (576, 64, 5120) (576,)
3 (864, 64, 5120) (864,)
4 (1152, 64, 5120) (1152,)
5 (1440, 64, 5120) (1440,)
6 (1728, 64, 5120) (1728,)
7 (2016, 64, 5120) (2016,)
8 (2304, 64, 5120) (2304,)
9 (2592, 64, 5120) (2592,)
fold number 1

1 (576, 64, 5120) (576,)
3 (864, 64, 5120) (864,)
4 (1152, 64, 5120) (1152,)
5 (1440, 64, 5120) (1440,)
6 (1728, 64, 5120) (1728,)
7 (2016, 64, 5120) (2016,)
8 (2304, 64, 5120) (2304,)
9 (2592, 64, 5120) (2592,)


KeyboardInterrupt: 

In [2]:
# Import APIs
import numpy as np
import scipy.io
import mne.io

path = '/home/hjkwon/Downloads/normal&drowsiness dataset/1/'
data = mne.io.read_raw_cnt(path + 'Normal state.cnt', eog=(), misc=(), ecg=(), emg=(),
                          data_format='auto', date_format='mm/dd/yy', preload=False, stim_channel=False, verbose=None)

/home/hjkwon/anaconda3/lib/python3.7/site-packages/numba/decorators.py:146: RuntimeWarning: Caching is not available when the 'parallel' target is in use. Caching is now being disabled to allow execution to continue.
  warnings.warn(msg, RuntimeWarning)
<ipython-input-2-d318b622b6c3>:8: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  data_format='auto', date_format='mm/dd/yy', preload=False, stim_channel=False, verbose=None)
<ipython-input-2-d318b622b6c3>:8: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  data_format='auto', date_format='mm/dd/yy', preload=False, stim_channel=False, verbose=None)


In [42]:
print(data.get_data().shape)

(40, 600880)


In [1]:
print(data.ch_names)
print(data.to_data_frame)
# print(data.times)

NameError: name 'data' is not defined